#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
from tqdm import tqdm
from warnings import warn
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pickle

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

## Huangrui's Dataset

In [2]:
default_mappings = {
    'label_maps': [{1.0: 'Did recid.', 0.0: 'No recid.'}],
    'protected_attribute_maps': [{0.0: 'Male', 1.0: 'Female'},
                                 {1.0: 'Caucasian', 0.0: 'Not Caucasian'}]
}
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
class CompasDataset(StandardDataset):
    """ProPublica COMPAS Dataset.

    See :file:`aif360/data/raw/compas/README.md`.
    """

    def __init__(self, label_name='Y', favorable_classes=[1],
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[],
                 features_to_drop=[], na_values=[],
                 custom_preprocessing=None,
                 metadata=default_mappings,
                 path='./Huangrui/recidivism/recidivism_test1.csv'):

    
        df = pd.read_csv(path,index_col=False)
        df.rename(columns={'Probationerssex_Female': 'sex'}, inplace=True)
        df.drop(["Probationerssex_Male","Probationerssex_Notascertained"], axis=1, inplace=True)
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)
        
        super(CompasDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)


#### Load dataset and specify options

In [3]:
## import dataset
dataset_used = "compas" # "adult", "german", "compas"

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
        
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

#### Split into train, test and validation

In [76]:
K=5
budget = 1 #0.01, 0.1, 1

In [77]:
dataset_orig_train= CompasDataset(path="./Huangrui/recidivism/recidivism_train{}.csv".format(K),
                                       protected_attribute_names=["sex"],
                                       privileged_classes=[[1]])

C:\TEMP\ipykernel_14808\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_14808\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_14808\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

In [78]:
dataset_orig_test= CompasDataset(path="./Huangrui/recidivism/recidivism_test{}.csv".format(K),
                                       protected_attribute_names=["sex"],
                                       privileged_classes=[[1]])

C:\TEMP\ipykernel_14808\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_14808\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
C:\TEMP\ipykernel_14808\3106683521.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

In [79]:
# Get the dataset and split into train and test
if budget == 0.01:
    #for budget = 0.01 use seed 43 
    dataset_orig_train,_ = dataset_orig_train.split([budget ], shuffle=True)
else:
    dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)

#### Clean up training data and display properties of the data

In [80]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(9317, 528)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['Threedigitoffensecode_Allotherexceptcannabis', 'Threedigitoffensecode_Armedrobbery', 'Threedigitoffensecode_Assault', 'Threedigitoffensecode_Attemptedhomicide', 'Threedigitoffensecode_Burglary', 'Threedigitoffensecode_Burglaryinvolvingvict', 'Threedigitoffensecode_Cannabis', 'Threedigitoffensecode_Drugpossession', 'Threedigitoffensecode_Drugtrafficking', 'Threedigitoffensecode_Escape', 'Threedigitoffensecode_ForcibleRape', 'Threedigitoffensecode_Forgery', 'Threedigitoffensecode_Fraud', 'Threedigitoffensecode_Homicides', 'Threedigitoffensecode_Manslaughter', 'Threedigitoffensecode_Miscellaneous', 'Threedigitoffensecode_Motorvehicularoffenses', 'Threedigitoffensecode_Murder', 'Threedigitoffensecode_Narcotics,cocaine', 'Threedigitoffensecode_Notascertained', 'Threedigitoffensecode_Otherstructure', 'Threedigitoffensecode_Possessionstolenproperty', 'Threedigitoffensecode_Rape', 'Threedigitoffensecode_Residential', 'Threedigitoffensecode_Robbery', 'Threedigitoffensecode_Sexrelatedoffenses'

#### Metric for original training data

In [81]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.149242


### Train classifier on original data

In [82]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()

lmod = pickle.load(open('experiments/recidivism'+str(K)+'_sex_bmodel.pkl','rb'))['clf']
y_train_pred = lmod.predict(X_train)

dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_train_pred.labels = y_train_pred.reshape(-1,1)
sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)

C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [83]:
y_train_pred

array([-0.17618501, -0.51549429, -0.08486126, ..., -0.0388194 ,
        1.35044478,  0.10097131])

#### Obtain scores for validation and test sets

In [84]:
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = scale_orig.transform(dataset_orig_test_pred.features)
y_test = dataset_orig_test_pred.labels
dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)

### Find the optimal parameters from the validation set

#### Best threshold for classification only (no fairness)

In [85]:
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = dataset_orig_train_pred.scores > class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
    
    classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                             dataset_orig_train_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    
    ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                       +classified_metric_orig_train.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)

Best balanced accuracy (no fairness constraints) = 0.7642
Optimal classification threshold (no fairness constraints) = 0.4753


#### Estimate optimal parameters for the ROC method

In [86]:
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)

In [87]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.2971
Optimal ROC margin = 0.0000


### Predictions from Validation Set

In [88]:
# Metrics for the test set
fav_inds = dataset_orig_train_pred.scores > best_class_thresh
dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

display(Markdown("#### train set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                unprivileged_groups, privileged_groups)

#### train set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.7642
Statistical parity difference = 0.2220
Disparate impact = 1.7904
Average odds difference = 0.1629
Equal opportunity difference = 0.2036
Theil index = 0.1283


In [89]:
# Transform the validation set
dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

display(Markdown("#### train set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                unprivileged_groups, privileged_groups)

#### train set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.5717
Statistical parity difference = 0.1548
Disparate impact = 1.2029
Average odds difference = 0.1078
Equal opportunity difference = 0.0471
Theil index = 0.0620


In [90]:
# Testing: Check if the metric optimized has not become worse
assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

### Predictions from Test Set

In [91]:
# Metrics for the test set
fav_inds = dataset_orig_test_pred.scores > best_class_thresh
dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

display(Markdown("#### Test set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.7562
Statistical parity difference = 0.1655
Disparate impact = 1.6272
Average odds difference = 0.1324
Equal opportunity difference = 0.1714
Theil index = 0.1536


In [92]:
# Metrics for the transformed test set
dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

display(Markdown("#### Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.5812
Statistical parity difference = 0.1143
Disparate impact = 1.1450
Average odds difference = 0.0792
Equal opportunity difference = 0.0243
Theil index = 0.0610


In [93]:
#自己计算error, 不是balanced accuracy！！！
metric_test_aft["Equal opportunity difference"]
print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))

The Error for the test dataset is 0.5103
The Equal opportunity difference for the test dataset is 0.02426
